In [1]:
import pandas
import numpy as np

In [2]:
df = pandas.read_csv('diabetes.csv')
data = df.values
labels = df.columns
# print(data)
# print(labels)
# print(type(labels))
# labels = list(labels)

In [3]:
def split_data(data,index,split):
    """
    Parameters:
    -----------
    rows:数据集
    index:列索引值
    split:在CART算法中是对应的最好分隔值
    
    Returns:
    ------- 
    left:左子树
    right：右子树
    """
    left = [row for row in data if row[index] >= split]
    right = [row for row in data if row[index] < split]
    return (left,right)

def Gini(data):
    """
    计算基尼系数
    
    Parameters:
    -----------
    data：一个数据集
    
    Returns:
    ------- 
    Gini

    """
    m=len(data)
    labelcount = {}
    Gini = 1
    for row in data:
        currentlabel = row[-1]
        if currentlabel not in labelcount.keys():
            labelcount[currentlabel] = 0
        labelcount[currentlabel]+=1
    for value in labelcount:
        Gini -= (value/m)**2
    return Gini    

def major_class(rows):
    """
    Parameters:
    -----------
    rows:数据集
    
    Returns:
    -------    
    result:分类值：0或1
    """
    results = {}
    for row in rows:
        value = row[-1]
        if value not in results:
            results[value] = 0
        results[value] += 1
    if results[1]>=results[0]:
        result = 1
    else:
        result = 0
    return result


def vote(classList):
    
    """
    找出样本数最多的类别
    Parameters:
    -----------
    classList：某一个数据集的最后一列
    
    Returns:
    ------- 
    result：多数样本的分类结果 0或1
    
    """
    classCount = {}
    for i in classList:
        if(i not in classCount.keys()):
            classCount[i] = 0
        classCount[i] += 1
    if classCount[1]>classCount[0]:
        result = 1
    else:
        result = 0
    return result


def chooseBestFeature(data):
    """
    通过Gini值选出最好的特征~
    Parameters:
    -----------
    data：平平无奇数据集
    
    Returns:
    ------- 
    best_split：最好的分隔值
    axis：最好的特征的列索引值

    """

    sorted_data = np.sort(data[:, :-1], axis=0)#不要最后一行 进行整理 axis=0表示按列排序
    sorted_data = sorted_data.T.tolist()#转置，且将数组转换为列表 相当于列表嵌套 写到一行

    
    max_impurity=0
    for row in range(len(sorted_data)):#row取了一行 因为是转置过的矩阵 所以其实是原来的一列 
        sorted_ = sorted_data[row]#取出了sorted_data中的第row个数组
        sorted_set = sorted(list(set(sorted_)))#对于数字去重 再整理  得到一个嵌套列表

        split = [(sorted_set[i] + sorted_set[i + 1]) / 2 for i in range(len(sorted_set)-2)]#依次计算了平均值  构成了一个比原来小2的列表
        for value in split:#对于这个平均值列表里的每一个平均值 进行分裂左右节点 
            left = data[np.where(data[:, row] <= value)[0], :]
            right = data[np.where(data[:, row] > value)[0], :]
            p1 = float(len(left)/len(data))
            p2 = float(len(right)/len(data))
            impurity = p1*Gini(left)+p2*Gini(right)#计算在该种分裂方式下 纯度）
            if impurity >= max_impurity:
                best_split = split
                axis = row
    return best_split,axis
        
def buildTree(data,labels):
    """
    Parameters:
    -----------
    data:数据集
    labels：对应属性
    
    Returns:
    ------- 
    tree：一个字典
    
    """
    labels=labels
    classList = [example[-1] for example in data]#1或0

    if(len(classList) == classList.count(classList[0])):#classlist中的第一个元素出现的次数等于classlist的长度 说明全部属于同一个类
        return classList[0]
   
    if(len(data[0]) == 1):#只剩一个特征属性
        majorClass = vote(classList)
        return majorClass
    
    best_split,axis = chooseBestFeature(data)#最优划分特征 的下标号
    best_feaLabel = labels[axis]#在labels中找出下标为axis的属性到底是啥  #好家伙好家伙  为什么这一行报错！！！！
    tree = {'best_feaLabel>best_split':{}}#子树的根的key是此次划分的最优特征名，value是再往下递归划分的子树
    tree[best_feaLabel>best_split][value] = buildTree(data,labels)
    return tree

In [ ]:
buildTree(data,labels)